In [ ]:
# check system specs

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print('Connected to a GPU')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

if ram_gb < 20:
  print('Not using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))
else:
  print('Using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))

Connected to a GPU
Using a high-RAM runtime: 89.6 gigabytes of available RAM


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# note: place shortcut to shared project folder in google drive root directory
%cd /content/gdrive/MyDrive/ml2_project

/content/gdrive/.shortcut-targets-by-id/1WHLBzPq6pt_F7mh3d3goIQl4MwYlTfIh/ml2_project


In [ ]:
# load models for comparison

from sentence_transformers import SentenceTransformer
import torch

# base_model = SentenceTransformer('all-MiniLM-L12-v2')
# base_model.save('./all-MiniLM-L12-v2')

base_model = SentenceTransformer('./models/all-MiniLM-L12-v2')
base_model = base_model.to(torch.device('cuda')) # use GPU

ft_128_model = SentenceTransformer('./models/finetune_mnr_128seqlength')
ft_128_model = ft_128_model.to(torch.device('cuda')) # use GPU

ft_510_model = SentenceTransformer('./models/finetune_mnr_510seqlength')
ft_510_model = ft_510_model.to(torch.device('cuda')) # use GPU

### Validation Methodology
Half the validation songs will be the correct pairs with similarity score 1. The other half will be random halves with similarity 0.

Reference: https://www.pinecone.io/learn/fine-tune-sentence-transformers-mnr/#compare-sentence-transformers

In [ ]:
lyrics_val = pd.read_csv('data/lyrics_val.csv',names = ['index','lyrics'])
lyrics_val = lyrics_val['lyrics']

In [ ]:
# to be split into correct halves
lyrics_val_pos = lyrics_val[:len(lyrics_val)//2]

# to be split into randome halves
lyrics_val_neg = lyrics_val[len(lyrics_val)//2:]

In [ ]:
from sentence_transformers import InputExample

In [ ]:
# prep positives

val_samples_pos = []
for song in lyrics_val_pos:
    song = str(song)
    half_1 = song[:len(song)//2]
    half_2 = song[len(song)//2:]
    val_samples_pos.append(InputExample(
        texts=[half_1, half_2],
        label=1
    ))

In [ ]:
# prep negatives

import random

val_samples_neg = []
for song in lyrics_val_neg:
    song = str(song)
    rand_index = random.randrange(len(lyrics_val_neg))
    other_song = str(lyrics_val_neg.iloc[rand_index])
    half_1 = song[:len(song)//2]
    half_2 = other_song[len(song)//2:]
    val_samples_neg.append(InputExample(
        texts=[half_1, half_2],
        label=0
    ))

In [ ]:
# combine

val_samples = val_samples_pos + val_samples_neg

In [ ]:
# create evaluator

from sentence_transformers.evaluation import BinaryClassificationEvaluator

evaluator_val = BinaryClassificationEvaluator.from_input_examples(
    val_samples, write_csv=False
)

In [ ]:
evaluator_val(base_model)

0.951259509783908

In [ ]:
evaluator_val(ft_128_model)

0.9898555396239681

In [ ]:
evaluator_val(ft_510_model)

0.9979744031786497

### Final Test
Choosing 128 sequence length model because it performs nearly as well as the 510 model with significantly less compute needed

In [ ]:
from sentence_transformers import InputExample
import random
from sentence_transformers.evaluation import BinaryClassificationEvaluator

In [ ]:
lyrics_test = pd.read_csv('data/lyrics_test.csv',names = ['index','lyrics'])
lyrics_test = lyrics_test['lyrics']

# to be split into correct halves
lyrics_test_pos = lyrics_test[:len(lyrics_test)//2]

# to be split into randome halves
lyrics_test_neg = lyrics_test[len(lyrics_test)//2:]

# prep positives

test_samples_pos = []
for song in lyrics_test_pos:
    song = str(song)
    half_1 = song[:len(song)//2]
    half_2 = song[len(song)//2:]
    test_samples_pos.append(InputExample(
        texts=[half_1, half_2],
        label=1
    ))

# prep negatives

test_samples_neg = []
for song in lyrics_test_neg:
    song = str(song)
    rand_index = random.randrange(len(lyrics_test_neg))
    other_song = str(lyrics_test_neg.iloc[rand_index])
    half_1 = song[:len(song)//2]
    half_2 = other_song[len(song)//2:]
    test_samples_neg.append(InputExample(
        texts=[half_1, half_2],
        label=0
    ))

# combine

test_samples = test_samples_pos + test_samples_neg

# create evaluator

evaluator_test = BinaryClassificationEvaluator.from_input_examples(
    test_samples, write_csv=False
)


In [ ]:
evaluator_test(ft_128_model)

0.9943836682301332